In [1]:
import os
from spark_utils import *
from pyspark.sql import functions as F

spark = create_spark_session()

bucket = 's3a://helenaudacitybucket'
load_pw = spark.read.parquet(os.path.join(bucket, 'processed_data', 'PW'))

calculate average salary

In [2]:
load_pw.groupBy("BUSINESS_NAME", "PRIMARY_WORKSITE_STATE", "PRIMARY_WORKSITE_COUNTY").\
agg(F.avg('PWD_WAGE_RATE').alias('mean_wage'), 
    F.count('PWD_WAGE_RATE').alias('count')
   ).orderBy('mean_wage', ascending=False).show()

+--------------------+----------------------+-----------------------+---------+-----+
|       BUSINESS_NAME|PRIMARY_WORKSITE_STATE|PRIMARY_WORKSITE_COUNTY|mean_wage|count|
+--------------------+----------------------+-----------------------+---------+-----+
|  McLaren Bay Region|                    MI|                    BAY| 280904.0|    1|
|Parkersburg Cardi...|                    WV|                   WOOD| 269173.0|    1|
|CENTRAL CALIFORNI...|                    CA|                 FRESNO| 257067.0|    1|
|       Rentlogic LLC|                    NY|               NEW YORK| 256776.0|    1|
|       LMC US Corp. |                    NY|               NEW YORK| 256776.0|    1|
|McIntosh Trail Co...|                    GA|                  HENRY| 255362.0|    1|
|Mcintosh Trail Co...|                    GA|                  HENRY| 255362.0|    1|
|BDT CAPITAL PARTN...|                    NY|               NEW YORK| 254405.0|    1|
|Argus Information...|                    NY|         

In [3]:
load_lca = spark.read.parquet(os.path.join(bucket, 'processed_data', 'LCA'))

In [4]:
load_lca.groupBy("EMPLOYER_NAME", "WORKSITE_STATE", "WORKSITE_COUNTY").\
agg(F.avg('ANNUAL_INCOME').alias('mean_wage'), 
    F.count('ANNUAL_INCOME').alias('count')
   ).orderBy('mean_wage', ascending=False).show()

+--------------------+--------------+---------------+--------------------+-----+
|       EMPLOYER_NAME|WORKSITE_STATE|WORKSITE_COUNTY|           mean_wage|count|
+--------------------+--------------+---------------+--------------------+-----+
|THE NEW YORK STEM...|            NY|       NEW YORK|              6.05E9|    1|
|EZEN COMPUTER SER...|            IL|         DUPAGE|           4.77775E9|    1|
|      TEKPLANT, INC.|            NY|    WESTCHESTER|1.5334023333333333E9|    3|
|Missouri State Un...|            MO|         GREENE|          1.069985E9|    2|
|     AssureCare, LLC|            OH|       HAMILTON|       4.752307789E8|   10|
|       Sutter Health|            CA|     SACRAMENTO|4.1023092609090906E8|   11|
|        NetApp, Inc.|            CA|    SANTA CLARA| 3.183381066363636E8|   22|
|Millennium Softwa...|            GA|         FULTON|              1.25E7|    1|
|The Solomon R. Gu...|            NY|       NEW YORK|           9536000.0|    1|
|Precision Psychia...|      

*Take home message*: Doctors' compensation can be really high.
Also, the maximum income we can see from prevailing wages dataset is generally lower than the LCA dataset.

By definition, The prevailing wage is used as a measure of the minimum allowable wage to be paid by employers seeking to employ a foreign national in H-1B status. It is a calculation of the average wage rate paid by employers to similarly-employed workers in substantially comparable jobs in the geographic area of intended employment. 

Probably Department of Labor removed some high-income outliers.

In [24]:
# alternatively, we can use

# view = 'PW'
# load_pw.createOrReplaceTempView(view)
# output = spark.sql("""
# SELECT BUSINESS_NAME, PRIMARY_WORKSITE_STATE, PRIMARY_WORKSITE_COUNTY, 
# AVG(PWD_WAGE_RATE) AS AVE_WAGE, COUNT(PWD_WAGE_RATE) AS COUNT
# FROM PW  
# GROUP BY BUSINESS_NAME, PRIMARY_WORKSITE_STATE, PRIMARY_WORKSITE_COUNTY
# ORDER BY AVE_WAGE DESC
# LIMIT 10
# """)
# output.limit(100).toPandas()